In [8]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import sys

In [9]:
def show_img(img):
    img = np.uint8(img)
    x,y = img.shape[0], img.shape[1]
    cv2.imshow("img", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [10]:
# 采集特征点
def get_Homo_matrix(img1, img2):
    orb = cv2.ORB_create(5000)
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)                 # 检测特征点
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)       # 暴力匹配
    matches = bf.match(des1, des2)
    # 选取前100个匹配度较高的点
    matches = sorted(matches, key=lambda x: x.distance)[:100]
    if verbose:
        show_img(cv2.drawMatches(img1, kp1, img2, kp2, matches, None, flags=2))

    pts1 = []
    pts2 = []
    for m in range(len(matches)):
        pts1.append(kp1[matches[m].queryIdx].pt) 
        pts2.append(kp2[matches[m].trainIdx].pt)
        
    # 单应性匹配
    H, mask = cv2.findHomography(np.float32(pts1), np.float32(pts2), cv2.RANSAC)
    
#     # 基础矩阵匹配
#     H, mask = cv2.findFundamentalMat(np.float32(pts1), np.float32(pts2), cv2.RANSAC) 

    if verbose:
        matches = np.array(matches)
        matches = matches[mask.ravel() == 1].tolist()
        show_img(cv2.drawMatches(img1, kp1, img2, kp2, matches, None, flags=2))
    
    return H, mask


In [11]:
# 仿射变换后的图像拼接
def stitch_images(src, dst):
    H, W = src.shape[0], src.shape[1]
    for i in range(H):
        for j in range(W):
            flag = (src[i, j, :] != np.array([0, 0, 0])) 
            if flag.any():
                dst[i, j] = src[i, j]
    return dst

In [12]:
# 去黑边
def remove_black_pad(img):
    H, W = img.shape[0], img.shape[1]
    
    left = 0
    for i in range(0, H):
        if (img[:, i, :] != np.array([0, 0, 0])).any():
            left = i
            break
            
    right = W
    for i in range(left, W):
        if (img[:, i, :] == np.array([0, 0, 0])).all():
            right = i
            break
    
    top = 0
    for i in range(0, H):
        if (img[i, :, :] != np.array([0, 0, 0])).any():
            top = i
            break
    
    bottom = H
    for i in range(top, H):
        if (img[i, :, :] == np.array([0, 0, 0])).all():
            bottom = i
            break
#     print(top,bottom,left,right)
    return img[top:bottom,left:right,:]

In [19]:
# 对应变换拼接两张图片
def two_img_stitch(res, next_img):
    HH, extra = get_Homo_matrix(next_img, res)
    # res 不动，以res为视角对cur_img进行当前视角下的仿射变换
    img_changed = cv2.warpPerspective(next_img, HH, (res.shape[1] + next_img.shape[1], res.shape[0] + next_img.shape[0]))
    if verbose:
        show_img(img_changed)
    res = stitch_images(res, img_changed)
    # 去黑边
    res = remove_black_pad(res)
    # 加padding，防止界外截断
    res = cv2.copyMakeBorder(res, padd, padd, padd, padd, cv2.BORDER_CONSTANT, 0)
    if verbose:
        show_img(res)
    return res

In [20]:
# 拼接图像队列，有固定视角
def imglist_stitch(imglist):
    img_count = len(imglist)
#     view_point = int(img_count/2)
    view_point = 0
    res = imglist[view_point].copy()
    
    for cur_img_idx in range(img_count):
        print("%d/%d" % (cur_img_idx + 1, img_count))
        if cur_img_idx!=view_point:
            cur_img = imglist[cur_img_idx]
            res = two_img_stitch(res, cur_img)
            
    res = remove_black_pad(res)    
    return res

In [21]:
# 拼接图像队列，以类似于haffman树的方式拼接
# 视角尽量往中间靠
def imglist_stitch_haff(imglist):
    
    img_count = len(imglist)
    mid_idx = (img_count/2.0)
    
    while True:
        if(img_count<=1):
            break
            
        new_stitch_imglist = []
        
        for cur_img_idx in range(0, img_count - 1, 2):
            print("%d/%d" % (cur_img_idx + 1, img_count))
            res = imglist[cur_img_idx].copy()
            next_img = imglist[cur_img_idx + 1].copy()
            # 以靠近中间视角的图像作为不动图，另一张图进行仿射变换
            if cur_img_idx < mid_idx:
                res, next_img = next_img, res
            res = two_img_stitch(res, next_img)
            new_stitch_imglist.append(res)

        # 单数图剩最后一张
        if(img_count % 2 == 1):
            print("%d/%d" % (img_count - 1, img_count))
            res = imglist[img_count - 1 - 1].copy()
            next_img = imglist[img_count - 1].copy()
            res = two_img_stitch(res, next_img)
            new_stitch_imglist.append(res)
            
        imglist = new_stitch_imglist.copy()
        img_count = len(imglist)
        mid_idx = int(img_count/2)
            
    res = remove_black_pad(imglist[0]) 
    
    return res

In [22]:
# 中文路径图像读取
def cv2_imread(filePath):
    cv2_img=cv2.imdecode(np.fromfile(filePath,dtype=np.uint8),-1)
    return cv2_img

In [23]:
# 中文路径图像写入
def cv2_imwrite(filePath, img):
    cv2.imencode('.jpeg', img)[1].tofile(filePath)

In [29]:
if __name__ == '__main__':

    file_path = "./图像拼接数据/旋转_缩放/1/"
#     file_path = "./data/"
    filename_list = glob.glob(file_path + "*.jpg")
    # 如果需要查看详细过程的话，建议缩小一定的图片比例
    verbose = False
    padd = 0
    # 图片缩放比例
    # 如果进行图像缩放，有可能会使得图像拼接失败
    img_scale = 1
    img_list = []
    for filename in filename_list:
#         img = cv2.imread(filename)
        img = cv2_imread(filename)
        x, y, _ = img.shape
        img = cv2.resize(img, (y // img_scale, x // img_scale))
        padd = max(padd,max(x // img_scale,y // img_scale))
        img = cv2.copyMakeBorder(img, padd, padd, padd, padd, cv2.BORDER_CONSTANT, 0)
        img_list.append(img)
    
    if(len(img_list) == 0):
        print("当前文件夹内没有图像！")
        sys.exit()
    
    combine = imglist_stitch_haff(img_list.copy())
#     cv2.imwrite(file_cudapath + "final_huff.jpeg", combine)
    cv2_imwrite(file_path + "final_huff.jpeg", combine)
    show_img(combine)    

1/2
